In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load dataset
data = pd.read_csv('combined_final1.csv')

# Drop irrelevant columns
data = data.drop(columns=['% Change', '% Change vs Average', 'Date'])

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[['Open', 'High', 'Low', 'Close']])


In [2]:
from tensorflow.keras.models import load_model
import numpy as np

# Load your pre-trained LSTM model
lstm_model = load_model('lstm_price_predictor.h5')

# Predict the next stock price using LSTM
def predict_future_prices(lstm_model, recent_data, scaler):
    scaled_recent_data = scaler.transform(recent_data)
    input_data = np.array([scaled_recent_data[-60:]])  # Last 60 time steps
    input_data = input_data.reshape((1, input_data.shape[1], input_data.shape[2]))
    
    predicted_price = lstm_model.predict(input_data)
    predicted_price = scaler.inverse_transform([[0, 0, 0, predicted_price[0][0]]])[0][3]
    return predicted_price


In [3]:
import gymnasium as gym
import numpy as np

class StockTradingEnv(gym.Env):
    metadata = {'render.modes': ['human', 'rgb_array']}

    def __init__(self, data, render_mode=None):
        super(StockTradingEnv, self).__init__()
        self.data = data
        self.render_mode = render_mode
        self.current_step = 0
        self.action_space = gym.spaces.Discrete(3)  # Buy, Sell, Hold
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(60, 4), dtype=np.float32)

    def reset(self, seed=None, options=None):
        self.current_step = 0
        if seed is not None:
            np.random.seed(seed)
        return self._next_observation(), {}

    def step(self, action):
        self.current_step += 1
        reward = self._take_action(action)
        done = self.current_step >= len(self.data) - 60
        obs = self._next_observation()
        return obs, reward, done, {}

    def _next_observation(self):
        return self.data[self.current_step:self.current_step + 60].values

    def _take_action(self, action):
        # Implement your trading logic here
        return 0

    def render(self, mode='human'):
        if self.render_mode == 'rgb_array':
            return np.zeros((400, 600, 3), dtype=np.uint8)
        else:
            pass


In [4]:
from stable_baselines3 import DDPG

# Load the DDPG model or train a new one
ddpg_model = DDPG.load('ddpg_stock_trader.zip')  # Ensure the model is saved correctly

# If you need to train it from scratch
# ddpg_model = DDPG('MlpPolicy', env, verbose=1)
# ddpg_model.learn(total_timesteps=10000)


In [7]:
obs, info = env.reset()
for step in range(len(data) - 60):
    # Get the recent market data (last 60 time steps)
    recent_data = data[step:step + 60]

    # Use LSTM to predict future price
    predicted_price = predict_future_prices(lstm_model, recent_data, scaler)

    # Use predicted price to make a decision (buy, sell, hold)
    action, _states = ddpg_model.predict(obs)

    # Execute the trade and move to the next state
    obs, reward, done, info = env.step(action)

    if done:
        print("End of simulation")
        break

env.close()


NameError: name 'env' is not defined

In [10]:
import gymnasium as gym
from gymnasium.wrappers import RecordVideo, RecordEpisodeStatistics
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from stable_baselines3 import DDPG
from sklearn.preprocessing import MinMaxScaler

# Load your trained LSTM model
lstm_model = load_model('lstm_price_predictor.h5')

# Load your dataset
data = pd.read_csv('C:\\Users\\RST\\Desktop\\Minor project\\combined_final1.csv')

# Drop irrelevant columns like Date
data = data.drop(columns=['% Change', '% Change vs Average', 'Date'])

# Fit the scaler on the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[['Open', 'High', 'Low', 'Close']])

# Define the function for LSTM-based price prediction
def predict_future_prices(lstm_model, recent_data, scaler):
    scaled_recent_data = scaler.transform(recent_data)
    input_data = np.array([scaled_recent_data[-60:]])  # Last 60 time steps
    input_data = input_data.reshape((1, input_data.shape[1], input_data.shape[2]))
    
    predicted_price = lstm_model.predict(input_data)
    predicted_price = scaler.inverse_transform([[0, 0, 0, predicted_price[0][0]]])[0][3]
    return predicted_price


# Custom stock trading environment
class StockTradingEnv(gym.Env):
    metadata = {'render.modes': ['human', 'rgb_array']}

    

    def __init__(self, data, render_mode=None):
        super(StockTradingEnv, self).__init__()
        self.data = data
        self.render_mode = render_mode
        self.current_step = 0
        self.action_space = gym.spaces.Discrete(3)  # Buy, Sell, Hold
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(60, 4), dtype=np.float32)

    def reset(self, seed=None, options=None):
        self.current_step = 0
        if seed is not None:
            np.random.seed(seed)
        return self._next_observation(), {}

    def step(self, action):
        self.current_step += 1
        reward = self._take_action(action)
        done = self.current_step >= len(self.data) - 60
        obs = self._next_observation()
        return obs, reward, done, {}

    def _next_observation(self):
        return self.data[self.current_step:self.current_step + 60].values

    def _take_action(self, action):
        # Implement your trading logic here
        return 0

    def render(self, mode='human'):
        if self.render_mode == 'rgb_array':
            return np.zeros((400, 600, 3), dtype=np.uint8)
        else:
            pass

# Initialize the DDPG model
ddpg_model = DDPG.load('ddpg_stock_trader.zip')

# Create the environment and wrap it
env = StockTradingEnv(data, render_mode='rgb_array')
env = RecordVideo(env, video_folder='videos/run_1', episode_trigger=lambda x: x % 100 == 0)
env = RecordEpisodeStatistics(env)

# Ensure the environment is initialized and accessible
obs, info = env.reset()

# Simulation loop
for step in range(len(data) - 60):
    # Get the recent market data (last 60 time steps)
    recent_data = data[step:step + 60]

    # Use LSTM to predict future price
    predicted_price = predict_future_prices(lstm_model, recent_data, scaler)

    # Use predicted price to make a decision (buy, sell, hold)
    action, _states = ddpg_model.predict(obs)

    # Execute the trade and move to the next state
    obs, reward, done, info = env.step(action)

    if done:
        print("End of simulation")
        break

# Close the environment properly
env.close()


C:\Users\RST\Desktop\Minor project\test_venv\lib\site-packages\gymnasium\wrappers\monitoring\video_recorder.py:178: UserWarning: WARN: Unable to save last video! Did you call close()?
  logger.warn("Unable to save last video! Did you call close()?")
C:\Users\RST\Desktop\Minor project\test_venv\lib\site-packages\gymnasium\wrappers\record_video.py:94: UserWarning: WARN: Overwriting existing videos at C:\Users\RST\Desktop\Minor project\Model 3\videos\run_1 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


1/1 [==============================] - 1s 1s/step


ValueError: Error: Unexpected observation shape (60, 4) for Box environment, please use (6,) or (n_env, 6) for the observation shape.

In [12]:
import gymnasium as gym
from gymnasium.wrappers import RecordVideo, RecordEpisodeStatistics
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from stable_baselines3 import DDPG
from sklearn.preprocessing import MinMaxScaler

# Load your trained LSTM model
lstm_model = load_model('lstm_price_predictor.h5')

# Load your dataset
data = pd.read_csv('C:\\Users\\RST\\Desktop\\Minor project\\combined_final1.csv')

# Drop irrelevant columns like Date
data = data.drop(columns=['% Change', '% Change vs Average', 'Date'])

# Fit the scaler on the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[['Open', 'High', 'Low', 'Close']])

# Define the function for LSTM-based price prediction
def predict_future_prices(lstm_model, recent_data, scaler):
    scaled_recent_data = scaler.transform(recent_data)
    input_data = np.array([scaled_recent_data[-60:]])  # Last 60 time steps
    input_data = input_data.reshape((1, input_data.shape[1], input_data.shape[2]))
    
    predicted_price = lstm_model.predict(input_data)
    predicted_price = scaler.inverse_transform([[0, 0, 0, predicted_price[0][0]]])[0][3]
    return predicted_price


# Custom stock trading environment
class StockTradingEnv(gym.Env):
    metadata = {'render.modes': ['human', 'rgb_array']}

    def __init__(self, data, lstm_model, scaler, render_mode=None):
        super(StockTradingEnv, self).__init__()
        self.data = data
        self.render_mode = render_mode
        self.lstm_model = lstm_model
        self.scaler = scaler
        self.current_step = 0
        self.action_space = gym.spaces.Discrete(3)  # Buy, Sell, Hold
        # Update observation space shape to be 1D (60 * 5 = 300 elements)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(300,), dtype=np.float32)

    def reset(self, seed=None, options=None):
        self.current_step = 0
        if seed is not None:
            np.random.seed(seed)
        return self._next_observation(), {}

    def step(self, action):
        self.current_step += 1
        reward = self._take_action(action)
        done = self.current_step >= len(self.data) - 60
        obs = self._next_observation()
        return obs, reward, done, {}

    def _next_observation(self):
        # Get the next 60 time steps from the data
        recent_data = self.data[self.current_step:self.current_step + 60]

        # Predict the future price using the LSTM model
        predicted_price = predict_future_prices(self.lstm_model, recent_data, self.scaler)

        # Add predicted price as a new feature in the observation
        obs = np.hstack((recent_data.values, np.full((60, 1), predicted_price)))  # Append predicted price as 5th column

        # Flatten the observation into a 1D array (60 * 5 = 300)
        return obs.flatten()

    def _take_action(self, action):
        # Implement your trading logic here
        return 0

    def render(self, mode='human'):
        if self.render_mode == 'rgb_array':
            return np.zeros((400, 600, 3), dtype=np.uint8)
        else:
            pass

# Initialize the DDPG model
ddpg_model = DDPG.load('ddpg_stock_trader.zip')

# Create the environment and wrap it
env = StockTradingEnv(data, lstm_model, scaler, render_mode='rgb_array')
env = RecordVideo(env, video_folder='videos/run_1', episode_trigger=lambda x: x % 100 == 0)
env = RecordEpisodeStatistics(env)

# Ensure the environment is initialized and accessible
obs, info = env.reset()

# Simulation loop
for step in range(len(data) - 60):
    # Use predicted price and market data to make a decision
    action, _states = ddpg_model.predict(obs)

    # Execute the trade and move to the next state
    obs, reward, done, info = env.step(action)

    if done:
        print("End of simulation")
        break

# Close the environment properly
env.close()


C:\Users\RST\Desktop\Minor project\test_venv\lib\site-packages\gymnasium\wrappers\monitoring\video_recorder.py:178: UserWarning: WARN: Unable to save last video! Did you call close()?
  logger.warn("Unable to save last video! Did you call close()?")
C:\Users\RST\Desktop\Minor project\test_venv\lib\site-packages\gymnasium\wrappers\record_video.py:94: UserWarning: WARN: Overwriting existing videos at C:\Users\RST\Desktop\Minor project\Model 3\videos\run_1 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


1/1 [==============================] - 1s 1s/step


ValueError: Error: Unexpected observation shape (300,) for Box environment, please use (6,) or (n_env, 6) for the observation shape.